# Лекция 3: Оптимизация выполнения кода, векторизация, Numba

__Автор: Сергей Вячеславович Макрушин__ e-mail: SVMakrushin@fa.ru 

Финансовый универсиет, 2020 г. 

При подготовке лекции использованы материалы:
* Документация к рассмотренным пакетам

V 0.3 17.09.2020

## Разделы: <a class="anchor" id="разделы"></a>
* [Профилирование](#профилирование)
* [Numba](#numba)
* [Векторизация](#векторизация)

-

* [к оглавлению](#разделы)

In [43]:
# загружаем стиль для оформления презентации
from IPython.display import HTML
from urllib.request import urlopen
html = urlopen("file:./lec_v1.css")
HTML(html.read().decode('utf-8'))

В процессе разработки кода и создания конвейеров обработки данных всегда присутствуют компромиссы между различными реализациями. В начале создания алгоритма забота о подобных вещах может оказаться контрпродуктивной. Согласно знаменитому афоризму Дональда Кнута: _«Лучше не держать в голове подобные “малые” вопросы производительности, скажем, 97 % времени: преждевременная 
оптимизация — корень всех зол»_.

Однако, как только ваш код начинает работать, часто бывает полезно заняться его производительностью. Иногда бывает удобно проверить время выполнения заданной команды или набора команд, а иногда — покопаться в состоящем из множества строк процессе и выяснить, где находится узкое место какого-либо сложного набора операций. 

# Профилирование <a class="anchor" id="профилирование"></a>
* [к оглавлению](#разделы)

<em class="df"></em> __Профилирование__ — сбор характеристик работы программы, таких как: 
* время выполнения отдельных фрагментов (например, функций)
* число верно предсказанных условных переходов
* число кэш-промахов
* объем используемой оперативной памяти 
* и т. д. 

Инструмент, используемый для анализа работы, называют __профайлером__ (profiler). Обычно профилирование выполняется в процессе оптимизации программы.

Магические функции IPython для профилирования:

* `%time` - длительность выполнения отдельного оператора;
* `%timeit` - длительность выполнения отдельного оператора при неоднократном повторе (может использоваться для обсепечения большей точности оценки);
* `%prun` - выполнение кода с использованием профилировщика;
* `%lprun` - пошаговое выполнение кода с применением профилировщика;
* `%memit` - оценка использования оперативной памяти для отдельного оператора;
* `%mprun` - пошаговое выполнение кода с применением профилировщика памяти.

Для работы с одной строкой кода использутся строчные магические команды (например, `%time`), для работы с целой ячейкой их блочные аналоги (например, `%%time`).

In [1]:
%time sum(range(100))

Wall time: 0 ns


4950

In [45]:
%timeit sum(range(100))

1.95 µs ± 1.71 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [2]:
%%timeit 

total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

507 ms ± 2.39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Команда `%timeit` выполняет оценку времени многократного выполнения фрагментов кода и автоматически подстраивает колчество повторов выполненеия под длительность работы функции.

Дольные приставки Си при измерении времени:
* 1 ms - 1 миллисекунда (мс): 1/1000 секунды
* 1 µs - 1 микросекунда (мкс): 1/1000 000 секунды
* 1 ns - 1 наносекунда (нс): 1/1000 000 000 секунды

In [10]:
%%time

total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

Wall time: 674 ms


Команда `%time` выполняет однократный запуск кода. В отличие от `%timeit` `%time` не выполняет специальных действий, предотвращающих системные вызовы, поэтому _часто выполнение того же  кода с замером `%time` происходит несколько медленнее чем с `%timeit`_.

----

Задача: за разумное время найти дубликаты в списке фильмов, содержащемся в файле `tmdb_5000_credits.csv` (размер: 4803 строки).

In [5]:
import csv

def read_movies(src, skip_header=True, title_column_ind=1):    
    '''
    Parameters:
        src (String): имя файла с фильмами
        skip_header (bool, optional): Пропускать ли заголовок?
        title_column_ind (int, optional): Столбец с названиями фильмов
    Returns:
        list: Список названий фильмов из файла (столбец title_column_ind в CSV)
    '''
    with open(src) as fd:
        csv_reader = csv.reader(fd, delimiter=',')
        movies = [row[title_column_ind] for row in csv_reader]
        if skip_header:
            movies = movies[1:]
        return movies

# with open('employee_birthday.txt') as csv_file:
#     csv_reader = csv.reader(csv_file, delimiter=',')
#     line_count = 0
#     for row in csv_reader:
#         if line_count == 0:
#             print(f'Column names are {", ".join(row)}')
#             line_count += 1
#         else:
#             print(f'\t{row[0]} works in the {row[1]} department, and was born in {row[2]}.')
#             line_count += 1
#     print(f'Processed {line_count} lines.')

In [6]:
movies = read_movies('tmdb_5000_credits.csv')
movies[:5], type(movies), len(movies)

(['Avatar',
  "Pirates of the Caribbean: At World's End",
  'Spectre',
  'The Dark Knight Rises',
  'John Carter'],
 list,
 4803)

In [7]:
# 1я попытка:

def is_duplicate(needle, haystack):
    for movie in haystack:
        if needle.lower() == movie.lower():
            return True
    return False

def find_duplicate_movies(src='tmdb_5000_credits.csv'):
    movies = read_movies(src)
    duplicates = []
    while movies:
        movie = movies.pop()
        if is_duplicate(movie, movies):
            duplicates.append(movie)
    return duplicates

In [8]:
%%time

duplicates = find_duplicate_movies()
print(duplicates)

['Batman', 'Out of the Blue', 'The Host']
Wall time: 2.86 s


In [5]:
%%prun
# ищем "бутылочное горлышко":

duplicates = find_duplicate_movies()

Магическая функция %prun выдает результаты в формате, принятом у модулей профайлеров profile и cProfile (см. https://docs.python.org/3/library/profile.html ).

Столбцы содержат следующую информацию:
* __ncalls__ - количество вызовов функций (если дано 2 значения, например `3/1`, то это означает, что функция вызывалась рекурсивно (первое число - общее количество вызовов, второе - количество primitive call (вызовов которые не были порождены рекурсией))
* __tottime__ - общее количество времени, провдеенное в данной функции ( _ИСКЛЮЧАЯ время проведенное в вызовах подфункций_ )
* __percall__ - tottime / ncalls
* __cumtime__ - общее количество времени, провдеенное в данной функции ( _ВКЛЮЧАЯ время проведенное в вызовах подфункций_ ), это значение корректно расчитывается и для рекурсивных вызовов функций
* __percall__ - cumtime / primitive calls 
* __filename:lineno(function)__ - расположение функции

In [6]:
# дополнительная информация по %prun :
%prun?

In [6]:
# исправляем очевидное "слабое место" - огромное количество вызовов lower()

def is_duplicate2(needle, haystack):
    for movie in haystack:
        if needle == movie:
            return True
    return False

def find_duplicate_movies2(src='tmdb_5000_credits.csv'):
    movies = [movie.lower() for movie in read_movies(src)]
    duplicates = []
    while movies:
        movie = movies.pop()
        if is_duplicate2(movie, movies):
            duplicates.append(movie)
    return duplicates

In [7]:
%%time

duplicates = find_duplicate_movies2()
print(duplicates)

['batman', 'out of the blue', 'the host']
Wall time: 1.31 s


Иногда больше пользы может принести построчный отчет профилировщика. Такая функциональность не встроена в язык Python или оболочку IPython, но можно установить пакет line_profiler, обладающий такой возможностью. 

Установка пакета line_profiler:

* с помощью pip: `$ pip install line_profiler`
* с помощью conda (в Anaconda Prompt): `$ conda install -c anaconda line_profiler`

Документация: https://github.com/pyutils/line_profiler#id2

Другой аспект профилирования — количество используемой операциями памяти. Это количество можно оценить с помощью еще одного расширения оболочки IPython — memory_profiler. 

Установки пакета memory_profiler:

* с помощью pip: `$ pip install memory_profiler`
* с помощью conda (в Anaconda Prompt): `conda install -c anaconda memory_profiler`

Документация: 

In [12]:
# загружаем функционал line_profiler в Jupyther:
%load_ext line_profiler

In [13]:
%reload_ext line_profiler

In [14]:
# загружаем функционал memory_profiler в Jupyther:
%load_ext memory_profiler

In [15]:
%lprun?

Docstring:
Execute a statement under the line-by-line profiler from the
line_profiler module.

Usage:
  %lprun -f func1 -f func2 <statement>

The given statement (which doesn't require quote marks) is run via the
LineProfiler. Profiling is enabled for the functions specified by the -f
options. The statistics will be shown side-by-side with the code through the
pager once the statement has completed.

Options:

-f <function>: LineProfiler only profiles functions and methods it is told
to profile.  This option tells the profiler about these functions. Multiple
-f options may be used. The argument may be any expression that gives
a Python function or method object. However, one must be careful to avoid
spaces that may confuse the option parser.

-m <module>: Get all the functions/methods in a module

One or more -f or -m options are required to get any useful results.

-D <filename>: dump the raw statistics out to a pickle file on disk. The
usual extension for this is ".lprof". These stat

Сохраняем профилируемый код в файл .py :

In [17]:
%%writefile mprun_demo.py
# v.3
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
    return total

Overwriting mprun_demo.py


In [18]:
# импортируем интересующую функцию из файла:
from mprun_demo import sum_of_lists

In [19]:
%memit sum_of_lists(5000)

peak memory: 45.44 MiB, increment: 0.82 MiB


In [20]:
%lprun -f sum_of_lists sum_of_lists(5000)

Timer unit: 1e-07 s

Total time: 0.0102222 s
File: e:\ДЗ\TOBD\TOBD\lectures\mprun_demo.py
Function: sum_of_lists at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def sum_of_lists(N):
     3         1         20.0     20.0      0.0      total = 0
     4         6        251.0     41.8      0.2      for i in range(5):
     5         5      97188.0  19437.6     95.1          L = [j ^ (j >> i) for j in range(N)]
     6         5       4757.0    951.4      4.7          total += sum(L)
     7         1          6.0      6.0      0.0      return total

Необходимо указать ей явным образом, какие функции 
мы хотели быть профилировать, например так:

`In[10]: %lprun –f sum_of_lists sum_of_lists(5000)`

Записываем интересующий код в файл и импортируем из него:

In [16]:
%%writefile fdm_v2.py

#v.1

import csv

def read_movies(src, skip_header=True, title_column_ind=1):    
    '''
    Parameters:
        src (String): имя файла с фильмами
        skip_header (bool, optional): Пропускать ли заголовок?
        title_column_ind (int, optional): Столбец с названиями фильмов
    Returns:
        list: Список названий фильмов из файла (столбец title_column_ind в CSV)
    '''
    with open(src) as fd:
        csv_reader = csv.reader(fd, delimiter=',')
        movies = [row[title_column_ind] for row in csv_reader]
        if skip_header:
            movies = movies[1:]
        return movies

# исправляем очевидное "слабое место" - огромное количество вызовов lower()

def is_duplicate2(needle, haystack):
    for movie in haystack:
        if needle == movie:
            return True
    return False

def find_duplicate_movies2(src='tmdb_5000_credits.csv'):
    movies = [movie.lower() for movie in read_movies(src)]
    duplicates = []
    while movies:
        movie = movies.pop()
        if is_duplicate2(movie, movies):
            duplicates.append(movie)
    return duplicates

Overwriting fdm_v2.py


In [17]:
from fdm_v2 import find_duplicate_movies2

In [18]:
%lprun -f find_duplicate_movies2 find_duplicate_movies2()

Более 80% времени выполняется проверка is_duplicate2(). Переработаем алгоритм, для оптимзиации этой проверки:

In [19]:
# исправляем очередное "слабое место": неоптимальную проверку дубликатов:

def find_duplicate_movies3(src='tmdb_5000_credits.csv'):
    duplicates = []
    unique = set()
    for movie in read_movies(src):
        movie = movie.lower()
        if movie in unique:
            duplicates.append(movie)
        else:
            unique.add(movie)
    return duplicates

In [20]:
%%time

duplicates = find_duplicate_movies3()
print(duplicates)

['the host', 'out of the blue', 'batman']
Wall time: 897 ms


In [21]:
# результат:
duplicates

['the host', 'out of the blue', 'batman']

Если планируется профилировать код модулей и скриптов на Python вне Jupyter то для профилирования может быть удобно  использовать следующую технику:
* Добавить перед интересующими функциями декоратор `@profile`
* Запуститить профилирование с помощью утилиты `kernprof`, пример прфилирования скрипта `primes.py`: `kernprof -l -v primes.py` 
* Подробнее см.:  https://dwinston.github.io/python-second-language/extras/profiling.html

На основе сProfile (лежащего в основе prun) можно сделать декоратор:

In [22]:
import cProfile, pstats, io


def profile(fnc):
    
    """A decorator that uses cProfile to profile a function"""
    
    def inner(*args, **kwargs):
        
        pr = cProfile.Profile()
        pr.enable()
        retval = fnc(*args, **kwargs)
        pr.disable()
        s = io.StringIO()
        sortby = 'cumulative'
        ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        return retval

    return inner

In [23]:
@profile
def find_duplicate_movies4(src='tmdb_5000_credits.csv'):
    duplicates = []
    unique = set()
    for movie in read_movies(src):
        movie = movie.lower()
        if movie in unique:
            duplicates.append(movie)
        else:
            unique.add(movie)
    return duplicates

In [24]:
find_duplicate_movies4()

         19395 function calls in 0.907 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.002    0.002    0.907    0.907 <ipython-input-23-b88ed622b642>:1(find_duplicate_movies4)
        1    0.000    0.000    0.904    0.904 <ipython-input-1-183cd92cc113>:3(read_movies)
        1    0.803    0.803    0.903    0.903 <ipython-input-1-183cd92cc113>:14(<listcomp>)
     4890    0.005    0.000    0.101    0.000 C:\ProgramData\Anaconda3\envs\pyTorch_1_6\lib\encodings\cp1251.py:22(decode)
     4890    0.096    0.000    0.096    0.000 {built-in method _codecs.charmap_decode}
     4803    0.001    0.000    0.001    0.000 {method 'lower' of 'str' objects}
     4800    0.000    0.000    0.000    0.000 {method 'add' of 'set' objects}
        1    0.000    0.000    0.000    0.000 {built-in method io.open}
        1    0.000    0.000    0.000    0.000 C:\ProgramData\Anaconda3\envs\pyTorch_1_6\lib\_bootlocale.py:11(getpreferr

['the host', 'out of the blue', 'batman']

----

# Numba <a class="anchor" id="numba"></a>
* [к оглавлению](#разделы)

__Numba__ - JIT компилятор с открытым исходным кодом, который компилирует подмножество кода Python и NumPy в быстрый машинный код.

* Официальная страница проекта: https://numba.pydata.org/
*  <em class="df"></em> __JIT-компиляция__ (Just-in-time compilation, компиляция «на лету»), динамическая компиляция (dynamic translation) — технология увеличения производительности программных систем, использующих байт-код, путём компиляции байт-кода в машинный код или в другой формат непосредственно во время работы программы. 

Преимущества и накладные расходы: 
* достигается высокая скорость выполнения по сравнению с интерпретируемым байт-кодом (сравнимая с компилируемыми языками)
* накладные расходы: увеличение потребления памяти (для хранения результатов компиляции) и дополнительные затраты времени на компиляцию на лету.

__Ускорение функций на Python__

* Numba компилирует функции Python в оптимзированный машинный код с использованием библиотеки для компиляции промышленного уровня __LLVM__ (https://ru.wikipedia.org/wiki/LLVM). Численные алгоритмы откомпелированные с помощью Numba могут достигать скорости сопоставимой с исполнением откомпилированного кода на C или FORTRAN.

* Numba обеспечивает удобство работы:
    * нет необходимости уходить от использования обычного интерпретатора Python
    * нет необходимости выполнять отдельную компиляцию кода
    * нет необходимости в установке компилятора C/C++
    * Дотстаточно использовать декораторы Numba для ваших функций, Numba выполнит все необходимы шаги автоматически.

__Разработан для научных вычислений__

* Numba спроектирована для работы с массивами и функциями NumPy. 
* Numba генерирует специализированный код для различных типов массивов и их размещения для оптимизации производительности.
* Специализированные декораторы могут созвать ufunc которые могут использоваться для распространения по массивам NumPy, также как это делают ufunc NumPy.
* Numba хорошо интегрирована с работой Jupyter notebooks для обеспечения интерактивных вычислений и с распеределнными вычислительными средами, такими как Dask и Spark.

__Выполняет распараллеливание ваших алгоритмов__

* Numba поддерживает Simplified Threading: может автоматически выполнять выражения для NumPy на нескольких ядрах CPU, что делает простым написание параллельных циклов.
* Numba поддерживает SIMD Vectorization: Numba может автоматически транслировать некоторые циклы в векторные инструкции для CPU, что может обеспечивать 2-4 кратный прирост производительности. Numba адаптируется к имеющимся возможностям CPU, определяя и используя поддержку таки SIMD возможностей CPU как SSE, AVX или AVX-512.
* Numba поддерживает ускорение вычислений на GPU: поддерживаются драйверы NVIDIA CUDA и AMD ROCm. Numba позволяет писать параллельные GPU алгоритмы полностью из Python.

__Переносимые результаты компиляции__
* Numba обеспечивает высокую производительность приложений на Python без сложностей бинарной компиляции и создания пакетов. Исходный код остается написан на чистом Python, а Numba обеспечивает его компиляцию на лету. Numba проходит тестирование на более чем 200 различных программно-апаратных конфигурациях.
* Numba поддерживает:
    * разные CPU: Intel and AMD x86, POWER8/9, ARM.
    * разные GPU: NVIDIA и AMD.
    * разные версии Python: Python 2.7, Python 3.4-3.7
    * разные операционные системы: Windows, macOS, Linux
* Бинарные поставки Numba доступны для большинства систем в виде паетов conda и wheel для инсталляции с помощью pip.

Документация и учебные материалы:

http://numba.pydata.org/numba-doc/latest/index.html


Основные возможности Numba:

* генерация кода "на лету" (во время импорта или во время исполнения, по выбору пользователя)
* генерация нативного кода для CPU (по умолчанию) или для GPU
* интеграция со стеком технологий Python для научных вычислений (на основе NumPy)

In [25]:
import numpy as np
import numba
from numba import jit, njit

In [26]:
# наивная реализация суммы квадратов элементов матрицы:

def sum_sq_2d(arr):
    m, n = arr.shape
    result = 0.0
    for i in range(m):
        for j in range(n):
            result += arr[i,j] ** 2
    return result

In [27]:
np.full((10, 10), 42.0)

array([[42., 42., 42., 42., 42., 42., 42., 42., 42., 42.],
       [42., 42., 42., 42., 42., 42., 42., 42., 42., 42.],
       [42., 42., 42., 42., 42., 42., 42., 42., 42., 42.],
       [42., 42., 42., 42., 42., 42., 42., 42., 42., 42.],
       [42., 42., 42., 42., 42., 42., 42., 42., 42., 42.],
       [42., 42., 42., 42., 42., 42., 42., 42., 42., 42.],
       [42., 42., 42., 42., 42., 42., 42., 42., 42., 42.],
       [42., 42., 42., 42., 42., 42., 42., 42., 42., 42.],
       [42., 42., 42., 42., 42., 42., 42., 42., 42., 42.],
       [42., 42., 42., 42., 42., 42., 42., 42., 42., 42.]])

In [28]:
arr = np.full((1000, 1000), 42.0)
arr[:3, :3], arr.shape

(array([[42., 42., 42.],
        [42., 42., 42.],
        [42., 42., 42.]]),
 (1000, 1000))

Время работы наивной реализации:

In [29]:
%%timeit
sum_sq_2d(arr)

697 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Время работы реализации с использованием NumPy:

In [30]:
%%timeit
np.sum(arr ** 2)

3.89 ms ± 137 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Как работает Numba:
1. Читает байткод Python для декорированной функции.
2. Собирает информацию о типах входных аргументов функций.
3. Анализирует и оптимизирует код.
4. Использует библиотеку для компиляции LLVM для генерации машинного кода фонкции для конкретного CPU.
5. Данный машинный код используется каждый раз при вызове данной функции (с аргументами тогоже типа).

Когда Numba даст хороший прирост производительности:
* Если код ориентирован на численные операции, в т.ч.:
    * активно использует NumPy
    * имеется много циклов (большое количество итераций)

Как получить байткод Python:

In [31]:
import dis

In [32]:
dis.dis(sum_sq_2d)

  4           0 LOAD_FAST                0 (arr)
              2 LOAD_ATTR                0 (shape)
              4 UNPACK_SEQUENCE          2
              6 STORE_FAST               1 (m)
              8 STORE_FAST               2 (n)

  5          10 LOAD_CONST               1 (0.0)
             12 STORE_FAST               3 (result)

  6          14 SETUP_LOOP              54 (to 70)
             16 LOAD_GLOBAL              1 (range)
             18 LOAD_FAST                1 (m)
             20 CALL_FUNCTION            1
             22 GET_ITER
        >>   24 FOR_ITER                42 (to 68)
             26 STORE_FAST               4 (i)

  7          28 SETUP_LOOP              36 (to 66)
             30 LOAD_GLOBAL              1 (range)
             32 LOAD_FAST                2 (n)
             34 CALL_FUNCTION            1
             36 GET_ITER
        >>   38 FOR_ITER                24 (to 64)
             40 STORE_FAST               5 (j)

  8          42 LOAD_FAST   

* Реализация __CPython__ (_не путайте с Cython !_ ) интерпретирует не непосредственно исходный код, а компилирует его в байт код и исполняет (интерпретирует) его с помощью виртуальной машины (см: https://en.wikipedia.org/wiki/CPython , https://ru.wikipedia.org/wiki/CPython ).
* Байткод Python хранится в автоматических создаваемых при компиляци файлах с расширением `pyc` в папках `__pycache__ ` находящихся в папках рядом с файлами с расширением `py` (кодом модулей и скриптов на Python).
* Байткод создается при комплияции "на лету" кода на Python исполняемого в первый раз. 
* Среда Python автоматически отслеживает актуальность байткода в файлах с расширением `pyc` и при необходимости выполняет их обновление.

Numba является хорошим выбором если ваш код численно ориентирован (выполняет много математических вычислений), много использует NumPy и/или имеет много циклов. В этом примере мы применим самый фундаментальный из JIT-декораторов Numba, @jit, чтобы попытаться ускорить некоторые функции.

Пример использования декоратора `@jit`. 

Декоратор `@jit` имеет два режим работы:
* режим `nopython`
    * Устанавливается параметром `nopython=True` или использованием декоратора `@njit`
    * Это рекомендуемый для использования и наиболее быстрый режим.
    * Приводит к компиляции кода функции практически не используещего интерпретатор Python.
* режим `object`

In [33]:
# Пример использования декоратора @jit с параметром nopython=True

@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def go_fast(a): # Function is compiled to machine code when called the first time
    trace = 0
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
    return a + trace              # Numba likes NumPy broadcasting

In [34]:
# наивная реализация суммы квадратов элементов матрицы:

@njit
def sum_sq_2d_jit(arr):
    m, n = arr.shape
    result = 0.0
    for i in range(n):
        for j in range(n):
            result += arr[i,j] ** 2
    return result

In [35]:
%%time
# во время первого запуска с данным типом параметров производистя компиляция функции:
sum_sq_2d_jit(arr)

Wall time: 1.03 s


1764000000.0

Время работы откомпилированной реализации:

In [36]:
%%timeit
sum_sq_2d_jit(arr)

998 µs ± 3.03 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Возможно просмотреть код LLVM, который был сгенирирован при компиляции:

In [37]:
sum_sq_2d_jit.inspect_types()

sum_sq_2d_jit (array(float64, 2d, C),)
--------------------------------------------------------------------------------
# File: <ipython-input-34-dc37b4251e51>
# --- LINE 3 --- 
# label 0

@njit

# --- LINE 4 --- 

def sum_sq_2d_jit(arr):

    # --- LINE 5 --- 
    #   arr = arg(0, name=arr)  :: array(float64, 2d, C)
    #   $0.2 = getattr(value=arr, attr=shape)  :: tuple(int64 x 2)
    #   $0.5 = exhaust_iter(value=$0.2, count=2)  :: tuple(int64 x 2)
    #   del $0.2
    #   $0.3 = static_getitem(value=$0.5, index=0, index_var=None)  :: int64
    #   $0.4 = static_getitem(value=$0.5, index=1, index_var=None)  :: int64
    #   del $0.5
    #   m = $0.3  :: int64
    #   del m
    #   del $0.3
    #   n = $0.4  :: int64
    #   del $0.4

    m, n = arr.shape

    # --- LINE 6 --- 
    #   $const0.6 = const(float, 0.0)  :: float64
    #   result = $const0.6  :: float64
    #   del $const0.6
    #   jump 14
    # label 14

    result = 0.0

    # --- LINE 7 --- 
    #   jump 16
    # labe

---

# Векторизация <a class="anchor" id="векторизация"></a>
* [к оглавлению](#разделы)

 __Векторизация__ позволяет записывать применение функции для перобразования множества значений (вектора) за одну операцию. 
 
 Векторизация позвоялет:
 * писать более компактный и выразительный код
 * оптимизировать выполенние векторных операций по сравнению с применением циклов за счет специальных оптимизаций, в т.ч. за счет использования специальных возможностей процессоров, многие из которых поддерживают векторные операции на аппаратном уровне.
 
В контексте высокоуровневых языков, таких как Python, теримн векторизация означает использование оптимизированного заранее откомпилированного кода, написанного на низкоуровневом языке (например C) для выполнения математических операций над множестовм значений (вектором, массивом (в т.ч. многомерным)). Это делается вместо явного итерирования по данным на исходном высокоуровневом языке (например с помощью циклов Python).

* Пример решения задачи на скалярном языке (C):
 
```C
 for (i = 0; i < n; i++)
    for (j = 0; j < n; j++)
        a[i][j] += b[i][j];
```
Пример решения задачи на языке, поддерживающим векторные операции:
```Fortran
a = a + b
```
* Аналогичные примеры можно привести при переходе от кода на Python к использованию ufunc в NumPy.
* Современные языки, поддерживающие векторные операции:  APL, J, Fortran 90, Mata, MATLAB, Analytica, TK Solver (as lists), Octave, R, Cilk Plus, Julia, Perl Data Language (PDL), Wolfram Language, __библиотека NumPy в Python__.

* Но: реализованное в NumPy _множестов ufunc не обеспечивает решения всех возможных задач преобразования массивов_.

Пример для применения векторизации:

In [38]:
# подсчитываем количество нулей:
def count_zeros(v):
    result = 0
    while v:
        v, digit = divmod(v, 10)
        if digit == 0:
            result += 1
    return result

In [39]:
import numpy as np
import numpy.random

In [40]:
numpy.random.randint(0, 10000, 100)

array([7027,  240, 2561, 5594, 1736, 1499, 1482,  446, 7862, 7896,  387,
       4006, 6973, 9347, 5819, 4388, 9934, 8347, 2686, 4308, 4509, 6200,
       9719, 4616, 4537, 3670, 4004, 8705, 2948, 6077, 5645,  296, 6800,
       8526, 5255, 5552, 6130, 1684, 1797, 7796, 6711,  836, 9721, 9033,
       6675,  975, 3389, 9367, 9944, 8365, 9232, 8351, 9783, 9289, 8231,
       5944, 6157, 1965,  486, 7509, 6545, 2526, 2414, 7860, 5419, 5230,
       9749, 8666, 4062, 8534, 2009, 7220, 5667, 7932, 1149, 5564, 3627,
       3288, 3835, 7855, 7809, 4603, 1077, 1701, 6569, 1155,   94, 6213,
       4432, 8415, 4998, 4837, 5625, 6792, 4184, 2776,   84, 5009, 6182,
       8406])

In [41]:
vals = numpy.random.randint(0, 10000, 10000000)

In [42]:
%%time

z_count = 0
for v in vals:
    z_count += count_zeros(v)
print(z_count)

2890445
Wall time: 30.4 s


#### Векторизация в NumPy

__numpy.vectorize__ - это класс обобщенных функций, который позволяет создавать векторизованные функции в NumPy.
* `numpy.vectorize` позволяет определять векторизованные функции которые принимают массивы NumPy (или вложенные последовательности объектов) и возвращают массивы NumPy (единичные или кортежи).
* Конструктор класса выглядит следующим образом: `class numpy.vectorize(pyfunc, otypes=None, doc=None, excluded=None, cache=False, signature=None)`
* Ключевым аргументом является функция `pyfunc` - функция, которую требуется векторизовать.
* В результате применения конструктора `numpy.vectorize` появляется вызываемый (callable) объект типа `numpy.vectorize`, по сути это есть векторизованная функция.
* Векторизованная функция вызывает функцию `pyfunc` для элементов входных массиов аналогично функции `map` в Python, при этом применяются правила распространения (broadcasting) NumPy.

Подробнее о параметрах `numpy.vectorize`:
* `pyfunc : callable` - функция Python которую необходимо векторизовать
* `otypes : str or list of dtypes, optional` - тип выходных значений векторизованной функции. Может быть передан как строка с описанием кодов типов (typecode characters) или как список спецификаций типов данных. ( _См примеры_ )
* `doc : str, optional` - строка докумнетации функции, если передан None (значение по умолчанию) будет использована стрка документации функции `pyfunc`.
* `excluded : set, optional` - определение параметров по которым функция НЕ БУДЕТ векторизована, передается множество строк или чисел определяющих аргументы по именам параметров или по их позиции.
* `cache : bool, optional` - если True то при первом вызове кэшируется количество выходных значений, если параметр otypes не передан.
* `signature : string, optional` - обобщенная сигнатура функции, например `(m,n),(n)->(m)` для векторизованного матрично-векторного умножения. Если праметр передан `pyfunc` будет вызван для массивов с формой заданной размером соответсвующих измерений. По умолчанию считается что `pyfunc` принимает на вход скаляры и возвращает скаляры.

In [62]:
vcount_zeros = np.vectorize(count_zeros)

In [63]:
type(vcount_zeros)

numpy.vectorize

In [64]:
%%time

z_count = vcount_zeros(vals) # применение аналогично использованию ufunc 
print(np.sum(z_count))

2887290
Wall time: 11.2 s


In [65]:
# Тип возвращаемых значений определен автоматически:
z_count[:3], type(z_count[0])

(array([0, 1, 0]), numpy.int32)

In [66]:
vcount_zeros_f = np.vectorize(count_zeros, otypes=[float]) # явное задание возвращаемого типа

In [68]:
z_count_f = vcount_zeros_f(vals[:100])
z_count_f[:3], type(z_count_f[0])

(array([0., 1., 0.]), numpy.float64)

Пример использования параметра `excluded`:

In [69]:
# Расчет значения полинома с коэффциентами p для значения x:
def mypolyval(p, x):
    _p = list(p)
    res = _p.pop(0)
    while _p:
        res = res*x + _p.pop(0)
    return res

In [70]:
# При векторизации исключаем параметр p из параметров, по которым проводится векторизация:
vpolyval = np.vectorize(mypolyval, excluded=['p'])

In [72]:
vpolyval(p=[1, 2, 3], x=[0, 1])

array([3, 6])

In [73]:
vpolyval(p=[1, 2, 3], x=np.linspace(-1, 1, 100))

array([2.        , 2.00040812, 2.00163249, 2.00367309, 2.00652995,
       2.01020304, 2.01469238, 2.01999796, 2.02611978, 2.03305785,
       2.04081216, 2.04938272, 2.05876951, 2.06897255, 2.07999184,
       2.09182736, 2.10447913, 2.11794715, 2.1322314 , 2.1473319 ,
       2.16324865, 2.17998163, 2.19753086, 2.21589634, 2.23507805,
       2.25507601, 2.27589022, 2.29752066, 2.31996735, 2.34323028,
       2.36730946, 2.39220488, 2.41791654, 2.44444444, 2.47178859,
       2.49994898, 2.52892562, 2.5587185 , 2.58932762, 2.62075298,
       2.65299459, 2.68605244, 2.71992654, 2.75461688, 2.79012346,
       2.82644628, 2.86358535, 2.90154066, 2.94031221, 2.97990001,
       3.02030405, 3.06152433, 3.10356086, 3.14641363, 3.19008264,
       3.2345679 , 3.2798694 , 3.32598714, 3.37292113, 3.42067136,
       3.46923783, 3.51862055, 3.56881951, 3.61983471, 3.67166616,
       3.72431385, 3.77777778, 3.83205795, 3.88715437, 3.94306703,
       3.99979594, 4.05734109, 4.11570248, 4.17488011, 4.23487

#### Обобщенная сигнатура функции 


<b class="b n">Имеется потребность проводить векторизацию не только скалярных функций (принимающих в качестве аргументов один или несколько (фиксированное число!) скалярных аргументов и возвращающая одно значение), но и "векторных"(в нотации NumPy - работающих с массивами ndarray или анлогоичными структурами) функций.</b>
* В результате векторизации векторные функции могут эффективно (в смысле компактности записи и эффективности вычислений) применяться для массивов бОльших разменостей.
* Для реализации этого механизма конструктору `numpy.vectorize` необходимо передать информацию о том какая векторная структура у входных параметров и выходных значений. Это делается с помощью передачи _обобщенной сигнатуры функции_ через параметр `signature`.

__Обобщенная сигнатура функции__ (generalized ufunc signature) определяет как размерности каждого из входных/выходных массивов разбиваются на размерности относящиеся к ядру (т.е. становятся параметрами единичного вызова векторизуемой функции `pyfunc`) и на размерности, использующиеся для векторизации.

Основные парвила:
* каждое измерение в сигнатуре соотносится с измерениями соответствующих передаваемых массивов (соответствие строится начиная с конца кортежа, определяющего форму (shape) предаваемого массива).
* Измерения ядра, которым присвоены одинаковые имена, должны точно совпадать по размерам, в этом случае распространение (bradcasting) не производится.
* При применении векторизации измерения ядра убираются из всех входов, а для остающиеся измерений выполняется бродкастинг для выполнения итераций по ним в рамках работы векторизации.

Примеры обобщенных сигнатур различных функций:

<table border="1" class="docutils">
<colgroup>
<col width="17%">
<col width="37%">
<col width="46%">
</colgroup>
<thead valign="bottom">
<tr class="row-odd"><th class="head">Имя функции</th>
<th class="head">Сигнатуры</th>
<th class="head">Описание</th>
</tr>
</thead>
<tbody valign="top">
<tr class="row-even"><td>add</td>
<td><code class="docutils literal notranslate"><span class="pre">(),()-&gt;()</span></code></td>
<td>сложение, бинарная ufunc</td>
</tr>
<tr class="row-odd"><td>sum1d</td>
<td><code class="docutils literal notranslate"><span class="pre">(i)-&gt;()</span></code></td>
<td>сумма элементов вектора (reduction)</td>
</tr>
<tr class="row-even"><td>inner1d</td>
<td><code class="docutils literal notranslate"><span class="pre">(i),(i)-&gt;()</span></code></td>
<td>скалярное произведение двух векторов (vector-vector multiplication)</td>
</tr>
<tr class="row-odd"><td>matmat</td>
<td><code class="docutils literal notranslate"><span class="pre">(m,n),(n,p)-&gt;(m,p)</span></code></td>
<td>матричное умножение</td>
</tr>
<tr class="row-even"><td>vecmat</td>
<td><code class="docutils literal notranslate"><span class="pre">(n),(n,p)-&gt;(p)</span></code></td>
<td>умножения одномерного вектора (рассматривается как вектор-строка) на матрицу (vector-matrix multiplication)</td>
</tr>
<tr class="row-odd"><td>matvec</td>
<td><code class="docutils literal notranslate"><span class="pre">(m,n),(n)-&gt;(m)</span></code></td>
<td>умножение матрицы на одномерный вектора (рассматривается как вектор-столбец) (matrix-vector multiplication)</td>
</tr>
<tr class="row-even"><td>matmul</td>
<td><code class="docutils literal notranslate"><span class="pre">(m?,n),(n,p?)-&gt;(m?,p?)</span></code></td>
<td>функция, которая реализует все 4 варианта, рассмотренные выше</td>
</tr>
<tr class="row-odd"><td>outer_inner</td>
<td><code class="docutils literal notranslate"><span class="pre">(i,t),(j,t)-&gt;(i,j)</span></code></td>
<td>произведение двух матриц не по правилу "строка на столбец", а по правилу "строка на строку", при этом индекс второй строки определяет индекс столбца в котором будет помещено произведение в итоговой матрице
</td>
</tr>
<tr class="row-even"><td>cross1d</td>
<td><code class="docutils literal notranslate"><span class="pre">(3),(3)-&gt;(3)</span></code></td>
<td>Векторное произведение двух векторов размерности 3 ( https://ru.wikipedia.org/wiki/Векторное_произведение )</td>
</tr>
</tbody>
</table>


Документация:
* numpy.ufunc.signature: - https://numpy.org/doc/stable/reference/generated/numpy.ufunc.signature.html
* более подробно про обобщенную сигнатуру функции: https://numpy.org/doc/1.17/reference/c-api.generalized-ufuncs.html




Для реализованных в NumPy ufunc можно просмотреть их сигнатуру:

In [74]:
print(np.add.signature)

None


Отсутствие сигнатуры означет эквивалентно '(),()->()' (с поправкой на количество параметров функции).

In [76]:
np.linalg._umath_linalg.det.signature

'(m,m)->()'

In [77]:
def my_vecmat1(a, b):
    return np.sum(a * b)

In [78]:
my_vecmat1(np.arange(1,4),np.ones(3))

6.0

In [85]:
a1 = np.arange(1,13).reshape(3,4)
a1, a1.shape

(array([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]]),
 (3, 4))

In [80]:
np.ones(4)

array([1., 1., 1., 1.])

In [71]:
# (наверное) неожиданный результат:
my_vecmat1(a1, np.ones(4))

78.0

In [82]:
def my_vecmat2(a, b):
    return sum(x*y for x, y in zip(a, b))

In [83]:
my_vecmat2(np.arange(1,4),np.ones(3))

6.0

In [86]:
# (наверное) неожиданный результат:
my_vecmat2(a1, np.ones(3))

array([15., 18., 21., 24.])

In [87]:
# выполняем векторизацию векторной функции my_vecmat1 с описанием сигнатуры:
vmy_vecmat1 = np.vectorize(my_vecmat1, signature='(i),(i)->()')

In [88]:
a1

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])

In [89]:
np.ones(4)

array([1., 1., 1., 1.])

In [90]:
# применение векторизованной векторной функции:
vmy_vecmat1(a1, np.ones(4))

array([10., 26., 42.])

Что произошло: 
1. На входе: `(3, 4),(4)`
2. Сигнатура ядра: `(i),(i)->()`
3. Векторизаця: `(A, i),(i)->(A)`, где `A=3`, `i=4`
4. Результат: `(3)`

In [91]:
vmy_vecmat2= np.vectorize(my_vecmat2, signature='(i),(i)->()')

In [92]:
# указание правильной сигнатуры при векторизации позволило получить ожидаемый результат:
vmy_vecmat2(a1, np.ones(4))

array([10., 26., 42.])

In [93]:
b2 = np.vstack((np.ones((1,4)), np.full((1,4), 2), np.full((1,4), 3)))
b2, b2.shape

(array([[1., 1., 1., 1.],
        [2., 2., 2., 2.],
        [3., 3., 3., 3.]]),
 (3, 4))

In [94]:
a1

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])

In [95]:
vmy_vecmat1(a1, b2)

array([ 10.,  52., 126.])

Что произошло: 
1. На входе: `(3, 4),(3, 4)`
2. Сигнатура ядра: `(i),(i)->()`
3. Векторизаця: `(A, i),(A, i)->(A)`, где `A=3`, `i=4`
4. Результат: `(3)`

На самом деле это даже более простой случай, т.к. в первом случае для второго аргумента использовалось распространение (broadcasting)!

Кроме `numpy.vectorize` имется еще функция `numpy.frompyfunc` которая позволяет проеобразовывать скалярные функции Python в ufunc NumPy и использовать их с применением правил распространения.

In [96]:
# Пример numpy.frompyfunc:

oct_array = np.frompyfunc(oct, 1, 1)
oct_array(np.array((10, 30, 100)))

array(['0o12', '0o36', '0o144'], dtype=object)

#### Применение векторизации в Numba

In [97]:
# простой способ определить, какой тип Numba будет использовать для этих значений: 
numba.typeof(vals[0])

int32

In [98]:
# векторизация с помощью Numba (в явном виде передаем типы, компиляция происходит сразу):

numba_vcount_zeros = numba.vectorize(['int32(int32)'])(count_zeros)

In [99]:
%%time

z_count = numba_vcount_zeros(vals)
print(np.sum(z_count))

2887290
Wall time: 278 ms


In [100]:
# альтернативный способ:
from numba import vectorize

@vectorize(['int32(int32)'])
def numba2_vcount_zeros(v):
    result = 0
    while v:
        v, digit = divmod(v, 10)
        if digit == 0:
            result += 1
    return result

In [101]:
%%time

z_count = numba2_vcount_zeros(vals)
print(np.sum(z_count))

2887290
Wall time: 279 ms


#### Декоратор __@guvectorize__

<em class="qs"></em> А что, если ускорить рассчет перенеся все выполняемые вычисления в векторизованную функцию?

* декоратор `vectorize()` в Numba позволяет реализовывать скалярные ufuncs, которые обрабатывают один элемент за раз.
* декоратор `guvectorize()` идет на шаг вперед и позволяет векторизовать векторные ufunc которые обрабатывают массиывы определенных размеров и возвращают массивы определенных размеров. Типовой пример, это расчет медианы или фильтры свертки (convolution filter).

* в отличие от функций, полученных с помощью `vectorize()`, функции, полученные с помощью `guvectorize()`, не возвращают своих значенйи, вместо этого они получают массив для возвращаемого значения как аргумент функции и заполняют его во время работы. Это происходит из-за того что в реальности массив формируется с помощью механизмов NumPy и потом для него вызывается код сгенерированный с помощью Numba.


* Обобщенные универсальные функции (generalized universal functions) требуют описания сигнатуры размерностей для которых  реализована функция ядра. В Numba эта сигнатура определяется аналогично NumPy generalized-ufunc signature. (Не надо путать с сигнатурой типов, которую обычно требует Numba). Подробнее см.: https://numpy.org/doc/1.17/reference/c-api.generalized-ufuncs.html 

Рассмтрим, намриер сигнатуру матичного умножения `'(m,n), (n,p) -> (m,p)'`. Из нее видно, что:
* Ппервая с конца размерность первого аргумента и вторая с конца размерность второго аргумента должны совпадать (т.е. должно выполняться правило матричного умножения).
* Последние две размерности результата определяются соответствующими (по именам) размерностями первого и второго аргумента.
* Важно помнить: __соответствие__ реальных размерностей передаваемых массивов именам __сигнатуры строится начиная с конца кортежа, определяющего форму (shape) предаваемого массива__.

каждое измерение в сигнатуре соотносится с измерениями соответствующих передаваемых массивов (


При написании функции ядра для gufunc необходимо:
* Продумать сигнатуру (generalized-ufunc signature) функции.
* Рреализовывать функции соблюдая правила для размерностей вынесенные в сигнатуре
* Функция ядра для gufunc в Numba принимает в качестве параметров как сами аргументы функции так и переменную в которую будет помещаться результат работы функции
* Входной параметр для хранения результата является последним параметром функции.
* У функции не должно быть возвращаемых значений, все результаты должны сохраняться в последнем входном параметре функции.
* Последствия изменения значений других аргументов, кроме последнего, неопределены, поэтому полагаться на эти изменения нельзя.

In [102]:
# реализация ядра матричного умножения с сигнатурой '(m,n),(n,p)->(n,p)
def matmulcore(A, B, C):
    m, n = A.shape
    n, p = B.shape
    for i in range(m):
        for j in range(p):
            C[i, j] = 0
            for k in range(n):
                C[i, j] += A[i, k] * B[k, j]

* Обратите внимание как размерности `m`, `n` и `p` извлекаются из входных аргументов. 
* Размерность `n` извлекается дважды, для того чтобы подчеркнуть необходимость совпадения значений. На практике это действие не является необходимым.

Для построения generalized-ufunc из созданной функции ядра можно как явно вызывать функцию `numba.guvectorize` так и использовать декторатор `@guvectorize`. Интерфейс `numba.guvectorize` аналогичен функции `vectorize`, но дополнительно требует передачи сигнатуры.

In [106]:
from numba import guvectorize

In [107]:
gu_matmul = numba.guvectorize(['float32[:,:], float32[:,:], float32[:,:]', 'float64[:,:], float64[:,:], float64[:,:]' ],
                              '(m,n),(n,p)->(n,p)')(matmulcore)

The result is a gufunc, that can be used as any othe gufunc in NumPy. Broadcasting and type promotion rules are those on NumPy.

In [108]:
matrix_ct = 10000
gu_test_A = np.arange(matrix_ct * 2 * 4, dtype=np.float32).reshape(matrix_ct, 2, 4)
gu_test_B = np.arange(matrix_ct * 4 * 5, dtype=np.float32).reshape(matrix_ct, 4, 5)

In [109]:
%timeit gu_matmul(gu_test_A, gu_test_B)

887 µs ± 7.26 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Различия между функциями vectorize и guvectorize:
1. vectorize генерирует ufuncs, guvectorize генерирует generalized-ufuncs
2. В обоих случаях сигнатуры для типов входных аргументов и возвращаемых значений представлены в виде списка, но в функции vectorize для их определения используются сигнатуры, тогда как в guvectorize вместо этого используются списки типов, и последним специфириуется возвращаемое значение.
3. Для guvectorize необходимо передать сигнатуру NumPy generalized-ufunc signature. Эта сигнатура дложна соответствовать переданной сигнатуре типов.
4. Помните, что в guvectorize результат передается через последний параметр функции, тогда как в vectorize результат возвращается функцией ядра.

In [112]:
@guvectorize(['int32[:], int32'], '(n)->()')
def numba_vcount_zeros_arr(arr, result):
    result = 0    
    for i in range(arr.shape[0]):
        v = arr[i]
        while v:
            v, digit = divmod(v, 10)
            if digit == 0:
                result += 1       

In [113]:
%%time

z_count = numba_vcount_zeros_arr(vals)
print(z_count)

2887290
Wall time: 219 ms


* базовая реализаия - 21.1 s
* реализация с векторизацией - 6.61 s
* реализация с векторизацией на Numba - 105 ms
* реализация c guvectorize на Numba - 69.3 ms

In [168]:
# Итоговый прирост производительности:
21.1/0.0693

304.4733044733045